<a href="https://colab.research.google.com/github/AbdoCooder/csqa-cnn/blob/main/csqa_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abdelkaderabdocooder
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification


100%|██████████| 3.58G/3.58G [00:58<00:00, 65.4MB/s]


In [ ]:
import os
import shutil

# 1. Setup the path (same as before)
dataset_path = '/content/fruits-fresh-and-rotten-for-classification/dataset/train'

# 2. Create the two main categories
categories = ['Fresh', 'Rotten']
for category in categories:
    os.makedirs(os.path.join(dataset_path, category), exist_ok=True)

# 3. Loop through the existing folders and move files
for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)

    # Skip if it's not a folder or if it's one of our new target folders
    if not os.path.isdir(folder_path) or folder_name in categories:
        continue

    # Decide where to move the files based on the name
    destination = None
    if folder_name.lower().startswith('fresh'):
        destination = os.path.join(dataset_path, 'Fresh')
    elif folder_name.lower().startswith('rotten'):
        destination = os.path.join(dataset_path, 'Rotten')

    # Move the files if a destination was found
    if destination:
        print(f"Moving files from {folder_name} to {destination}...")
        for file_name in os.listdir(folder_path):
            src = os.path.join(folder_path, file_name)
            dst = os.path.join(destination, file_name)
            shutil.move(src, dst)

        # Optional: Remove the old empty folder to keep things clean
        os.rmdir(folder_path)

print("Reorganization complete!")

Moving files from freshapples to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Fresh...
Moving files from freshoranges to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Fresh...
Moving files from freshbanana to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Fresh...
Moving files from rottenoranges to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Rotten...
Moving files from rottenbanana to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Rotten...
Moving files from rottenapples to /content/fruits-fresh-and-rotten-for-classification/dataset/train/Rotten...
Reorganization complete!


In [ ]:
import tensorflow as tf

# 1. Define the path and parameters
train_path = '/content/fruits-fresh-and-rotten-for-classification/dataset/train'
batch_size = 32
img_height = 224
img_width = 224

# 2. Load the training data (80% of data)
train_ds = tf.keras.preprocessing.image_dataset_from_directory (
  train_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

# 3. Load the validation data (20% of data)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

# 4. Check the class names
class_names = train_ds.class_names
print("Classes found:", class_names)


Found 10901 files belonging to 2 classes.
Using 8721 files for training.
Found 10901 files belonging to 2 classes.
Using 2180 files for validation.
Classes found: ['Fresh', 'Rotten']


In [ ]:
# 1. Load the MobileNetV2 base model (without the top layer)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False
)

# 2. Freeze the base model (so we don't destroy pre-learned patterns)
base_model.trainable = False

# 3. Build the final model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2, activation='softmax') # 2 neurons for Fresh/Rotten
])

# 4. Check the structure
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         2,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("Model compiled and ready for training!")

Model compiled and ready for training!


In [ ]:
epochs = 5

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


Epoch 1/5
273/273 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - accuracy: 0.7651 - loss: 0.4857 - val_accuracy: 0.8972 - val_loss: 0.2552
Epoch 2/5
273/273 ━━━━━━━━━━━━━━━━━━━━ 565s 2s/step - accuracy: 0.8832 - loss: 0.2702 - val_accuracy: 0.9128 - val_loss: 0.2177
Epoch 3/5
273/273 ━━━━━━━━━━━━━━━━━━━━ 550s 2s/step - accuracy: 0.9076 - loss: 0.2297 - val_accuracy: 0.9216 - val_loss: 0.2093
Epoch 4/5
273/273 ━━━━━━━━━━━━━━━━━━━━ 594s 2s/step - accuracy: 0.9167 - loss: 0.2151 - val_accuracy: 0.9220 - val_loss: 0.1959
Epoch 5/5
273/273 ━━━━━━━━━━━━━━━━━━━━ 589s 2s/step - accuracy: 0.9191 - loss: 0.2022 - val_accuracy: 0.9275 - val_loss: 0.1886


In [15]:
"""IoT Conveyor Belt Fruit Classification Simulator.

This module simulates an IoT camera system that classifies fruits as fresh or rotten
using a Fined-Tuned pre-trained TensorFlow model.
"""

import os
import random
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Optional

import numpy as np
import tensorflow as tf


@dataclass
class SimulationStats:
    """Tracks statistics for the fruit classification simulation."""

    total_fruits:     int = 0
    rotten_detected:  int = 0
    fresh_detected:   int = 0

    @property
    def loss_rate(self) -> float:
        """Calculate the current loss rate as a percentage."""
        if self. total_fruits == 0:
            return 0.0
        return (self. rotten_detected / self.total_fruits) * 100

    def record(self, is_rotten: bool) -> None:
        """Record a classification result."""
        self.total_fruits += 1
        if is_rotten:
            self. rotten_detected += 1
        else:
            self.fresh_detected += 1


@dataclass
class ClassificationResult:
    """Holds the result of a single fruit classification."""

    image_path: str
    predicted_label: str
    confidence: float
    is_anomaly: bool

    @property
    def status(self) -> str:
        """Return a human-readable status string."""
        return "🚨 REJECTED (ANOMALY)" if self.is_anomaly else "✅ ACCEPTED"


class FruitClassifier:
    """Handles fruit image classification using a pre-trained model."""

    CLASS_NAMES = ('Fresh', 'Rotten')
    IMAGE_SIZE = (224, 224)

    def __init__(self, model:  tf.keras.Model):
        self.model = model

    def classify(self, image_path: str) -> ClassificationResult:
        """Classify a single fruit image.

        Args:
            image_path: Path to the fruit image file.

        Returns:
            ClassificationResult containing prediction details.
        """
        img_array = self._preprocess_image(image_path)
        predictions = self.model. predict(img_array, verbose=0)

        predicted_class_index = int(np.argmax(predictions[0]))
        predicted_label = self.CLASS_NAMES[predicted_class_index]
        confidence = float(100 * np.max(predictions[0]))

        return ClassificationResult(
            image_path=image_path,
            predicted_label=predicted_label,
            confidence=confidence,
            is_anomaly=(predicted_label == 'Rotten')
        )

    def _preprocess_image(self, image_path: str) -> np.ndarray:
        """Load and preprocess an image for model inference."""
        img = tf.keras.utils.load_img(image_path, target_size=self.IMAGE_SIZE)
        img_array = tf.keras.utils.img_to_array(img)
        return tf.expand_dims(img_array, 0)


class IoTConveyorSimulator:
    """Simulates an IoT camera system on a conveyor belt."""

    def __init__(
        self,
        classifier: FruitClassifier,
        source_folder: str,
        processing_delay: float = 1.0,
        camera_id: str = "CAM_01"
    ):
        self.classifier = classifier
        self.source_folder = Path(source_folder)
        self.processing_delay = processing_delay
        self.camera_id = camera_id
        self.stats = SimulationStats()

    def _get_image_files(self) -> list[str]:
        """Collect all image files from Fresh and Rotten folders."""
        image_files = []

        for category in FruitClassifier.CLASS_NAMES:
            category_path = self.source_folder / category
            if not category_path.exists():
                raise FileNotFoundError(f"Category folder not found: {category_path}")

            for file_path in category_path.iterdir():
                if file_path.suffix.lower() in ('.jpg', '.jpeg', '.png', '.bmp'):
                    image_files.append(str(file_path))

        if not image_files:
            raise ValueError(f"No image files found in {self.source_folder}")

        return image_files

    def _print_result(self, item_number: int, result: ClassificationResult) -> None:
        """Print classification result to console."""
        print(f"\n[{self.camera_id}] Item #{item_number} Detected...")
        print(f"   Analysis: {result.predicted_label} ({result.confidence:.2f}%)")
        print(f"   Action:   {result.status}")
        print(f"   📊 Current Loss Rate: {self.stats.loss_rate:.2f}%")

    def run(self, num_fruits:  int = 10) -> SimulationStats:
        """Run the conveyor belt simulation.

        Args:
            num_fruits: Number of fruits to process in this simulation run.

        Returns:
            SimulationStats containing the results of this run.
        """
        print(f"--- STARTING IOT CONVEYOR BELT SIMULATION ({num_fruits} items) ---")

        all_files = self._get_image_files()
        random.shuffle(all_files)

        if num_fruits > len(all_files):
            print(f"Warning:  Requested {num_fruits} items but only {len(all_files)} available.")
            num_fruits = len(all_files)

        for i in range(num_fruits):
            result = self.classifier.classify(all_files[i])
            self. stats.record(result.is_anomaly)
            self._print_result(i + 1, result)

            if self.processing_delay > 0:
                time.sleep(self.processing_delay)

        self._print_summary()
        return self.stats

    def _print_summary(self) -> None:
        """Print final simulation summary."""
        print("\n" + "=" * 50)
        print("SIMULATION COMPLETE")
        print(f"   Total Processed: {self.stats.total_fruits}")
        print(f"   Fresh:  {self.stats.fresh_detected}")
        print(f"   Rotten: {self.stats.rotten_detected}")
        print(f"   Final Loss Rate: {self.stats. loss_rate:.2f}%")
        print("=" * 50)


def main():
    """Run the IoT conveyor belt simulation."""
    # Configuration
    source_folder = '/content/fruits-fresh-and-rotten-for-classification/dataset/train'

    # Assumes `model` is already loaded in the environment
    # model = tf.keras.models.load_model('path/to/your/model')

    classifier = FruitClassifier(model)
    simulator = IoTConveyorSimulator(
        classifier=classifier,
        source_folder=source_folder,
        processing_delay=1.0
    )

    stats = simulator.run()


if __name__ == "__main__":
    main()


--- STARTING IOT CONVEYOR BELT SIMULATION (10 items) ---

[CAM_01] Item #1 Detected...
   Analysis: Rotten (74.05%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 100.00%

[CAM_01] Item #2 Detected...
   Analysis: Fresh (93.52%)
   Action:   ✅ ACCEPTED
   📊 Current Loss Rate: 50.00%

[CAM_01] Item #3 Detected...
   Analysis: Rotten (71.51%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 66.67%

[CAM_01] Item #4 Detected...
   Analysis: Rotten (99.88%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 75.00%

[CAM_01] Item #5 Detected...
   Analysis: Rotten (98.32%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 80.00%

[CAM_01] Item #6 Detected...
   Analysis: Rotten (91.43%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 83.33%

[CAM_01] Item #7 Detected...
   Analysis: Rotten (87.98%)
   Action:   🚨 REJECTED (ANOMALY)
   📊 Current Loss Rate: 85.71%

[CAM_01] Item #8 Detected...
   Analysis: Fresh (99.64%)
   Action:   ✅ ACCEPTED
   

In [16]:
!pip install -q -U google-generativeai

In [4]:
"""AI-Powered Quality Control Report Generator.

This module integrates with Google's Gemini API to generate professional
quality control reports based on fruit classification statistics.
"""

import os
from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum
from typing import Optional, Protocol
from google.colab import userdata

import google.generativeai as genai

# For Jupyter/Colab environments
try:
    from IPython.display import Markdown, display
    HAS_IPYTHON = True
except ImportError:
    HAS_IPYTHON = False


class SeverityLevel(Enum):
    """Classification of quality control severity levels."""

    ACCEPTABLE = "acceptable"
    WARNING = "warning"
    CRITICAL = "critical"

    @classmethod
    def from_loss_rate(cls, loss_rate: float) -> "SeverityLevel":
        """Determine severity level based on loss rate percentage."""
        if loss_rate < 5:
            return cls. ACCEPTABLE
        elif loss_rate < 15:
            return cls.WARNING
        return cls.CRITICAL


class StatsProtocol(Protocol):
    """Protocol defining required statistics interface."""

    total_fruits: int
    fresh_detected: int
    rotten_detected: int
    loss_rate: float


@dataclass
class QualityStats:
    """Quality control statistics container."""

    total_fruits: int
    fresh_detected:  int
    rotten_detected:  int

    @property
    def loss_rate(self) -> float:
        """Calculate loss rate as a percentage."""
        if self.total_fruits == 0:
            return 0.0
        return (self.rotten_detected / self.total_fruits) * 100

    @property
    def severity(self) -> SeverityLevel:
        """Get the severity level based on current loss rate."""
        return SeverityLevel.from_loss_rate(self.loss_rate)

def get_working_model():
    """Dynamically finds a working model from your account."""
    print("🔍 Searching for available models...")
    try:
        # List all models available to your API key
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                # We prefer Flash or Pro models
                if 'flash' in m.name:
                    return m.name
                if 'pro' in m.name and 'vision' not in m.name:
                     return m.name
        # Fallback to the first available text model
        return 'models/gemini-pro'
    except Exception as e:
        print(f"Warning: Could not list models ({e}). Defaulting to gemini-1.5-flash")
        return 'gemini-1.5-flash'

@dataclass
class ReportConfig:
    """Configuration for report generation."""

    facility_name: str = "Industrial Fruit Sorting Facility"
    acceptable_threshold: float = 5.0
    critical_threshold: float = 15.0
    model_name: str = get_working_model()
    temperature: float = 0.7
    max_output_tokens: int = 2048


class ReportGenerator(ABC):
    """Abstract base class for report generators."""

    @abstractmethod
    def generate(self, stats: StatsProtocol, config: ReportConfig) -> str:
        """Generate a quality control report."""
        pass


class GeminiReportGenerator(ReportGenerator):
    """Generates quality control reports using Google's Gemini API."""

    def __init__(self, api_key: Optional[str] = None):
        """Initialize the Gemini report generator.

        Args:
            api_key: Google API key.  If not provided, will look for
                     GOOGLE_API_KEY environment variable.

        Raises:
            ValueError:  If no API key is provided or found.
        """
        self.api_key = api_key or os.environ.get("GOOGLE_API_KEY")

        if not self.api_key:
            raise ValueError(
                "API key required. Either pass api_key parameter or set "
                "GOOGLE_API_KEY environment variable."
            )

        genai.configure(api_key=self.api_key)

    def _build_prompt(self, stats: StatsProtocol, config: ReportConfig) -> str:
        """Construct the prompt for the AI model."""
        severity = SeverityLevel.from_loss_rate(stats.loss_rate)

        return f"""
You are the Quality Assurance Manager at {config.facility_name}.
An IoT system has just finished scanning a batch of fruit with the following results:

--- DATA REPORT ---
- Total Items Scanned: {stats.total_fruits}
- Fresh Items: {stats.fresh_detected}
- Rotten Items (Rejections): {stats.rotten_detected}
- LOSS RATE: {stats.loss_rate:.2f}%
- SEVERITY LEVEL: {severity.value. upper()}
-------------------

Thresholds:
- Acceptable:  < {config.acceptable_threshold}%
- Warning: {config.acceptable_threshold}% - {config.critical_threshold}%
- Critical: > {config.critical_threshold}%

Please generate a professional "Quality Control Incident Report" in Markdown format.
The report must include:

1. **Executive Summary**: Brief overview of the production run status.
2. **Severity Assessment**:  Classification based on the loss rate with explanation.
3. **Statistical Analysis**: Breakdown of the numbers with context.
4. **Root Cause Analysis**: Three specific probable causes for this level of rot
   (e.g., storage humidity, transport delays, harvest timing, supply chain issues).
5. **Immediate Actions**: Recommended operational steps to address the issue.
6. **Preventive Measures**: Long-term recommendations to prevent recurrence.
7. **Next Steps**: Follow-up actions and timeline.

Format the report professionally with clear sections and bullet points.
"""

    def generate(self, stats: StatsProtocol, config: Optional[ReportConfig] = None) -> str:
        """Generate a quality control report using Gemini.

        Args:
            stats: Statistics from the fruit classification simulation.
            config: Optional configuration for report generation.

        Returns:
            Generated report as a Markdown string.

        Raises:
            ConnectionError: If unable to connect to the Gemini API.
            RuntimeError: If the API returns an error response.
        """
        config = config or ReportConfig()

        try:
            model = genai.GenerativeModel(
                model_name=config.model_name,
                generation_config={
                    "temperature": config. temperature,
                    "max_output_tokens": config.max_output_tokens,
                }
            )

            prompt = self._build_prompt(stats, config)
            response = model.generate_content(prompt)

            if not response.text:
                raise RuntimeError("Empty response received from Gemini API")

            return response.text

        except genai.types.BlockedPromptException as e:
            raise RuntimeError(f"Prompt was blocked by safety filters: {e}")
        except genai.types.StopCandidateException as e:
            raise RuntimeError(f"Generation stopped unexpectedly: {e}")
        except Exception as e:
            raise ConnectionError(f"Failed to connect to Gemini API: {e}")


class QualityReportManager:
    """Manages quality report generation and display."""

    def __init__(self, generator: ReportGenerator):
        self.generator = generator
        self._last_report:  Optional[str] = None

    def create_report(
        self,
        stats: StatsProtocol,
        config: Optional[ReportConfig] = None,
        verbose: bool = True
    ) -> str:
        """Create a quality control report.

        Args:
            stats: Statistics from the fruit classification.
            config: Optional report configuration.
            verbose: Whether to print progress messages.

        Returns:
            Generated report as a string.
        """
        if verbose:
            print("🤖 Consulting the AI Quality Manager...")
            print(f"   Processing {stats.total_fruits} items with "
                  f"{stats.loss_rate:.2f}% loss rate...")

        self._last_report = self.generator.generate(stats, config)

        if verbose:
            print("✅ Report generated successfully!")

        return self._last_report

    def display_report(self, report: Optional[str] = None) -> None:
        """Display the report in the appropriate format.

        Args:
            report: Report to display.  Uses last generated report if not provided.
        """
        report = report or self._last_report

        if not report:
            print("No report available.  Generate one first using create_report().")
            return

        if HAS_IPYTHON:
            display(Markdown(report))
        else:
            print(report)

    def save_report(self, filepath: str, report: Optional[str] = None) -> None:
        """Save the report to a file.

        Args:
            filepath: Path to save the report.
            report: Report to save.  Uses last generated report if not provided.
        """
        report = report or self._last_report

        if not report:
            raise ValueError("No report available to save.")

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(report)

        print(f"📄 Report saved to:  {filepath}")


def create_test_stats(
    total:  int = 100,
    rotten: int = 40
) -> QualityStats:
    """Create test statistics for demonstration purposes."""
    return QualityStats(
        total_fruits=total,
        fresh_detected=total - rotten,
        rotten_detected=rotten
    )


# --- MAIN EXECUTION ---
def main():
    """Run the quality report generation."""
    # Option 1: Set API key via environment variable (recommended)
    # export GOOGLE_API_KEY="your-api-key-here"

    # Option 2: Pass API key directly (less secure, avoid in production)
    # generator = GeminiReportGenerator(api_key="your-api-key-here")
    # This retrieves the secret saved in Collab
    api_key = userdata.get('GOOGLE_API_KEY')

    # Set it as an environment variable so the Gemini library can find it automatically
    os.environ["GOOGLE_API_KEY"] = api_key
    try:
        generator = GeminiReportGenerator()
    except ValueError as e:
        print(f"⚠️  Configuration Error: {e}")
        print("\nTo use this module, set your API key:")
        print("  Option 1: Set GOOGLE_API_KEY environment variable")
        print("  Option 2: Pass api_key to GeminiReportGenerator()")
        return

    manager = QualityReportManager(generator)

    # Use existing stats if available, otherwise create test data
    # This allows seamless integration with the IoT simulator
    if 'stats' in globals():
        quality_stats = stats  # type: ignore
        print("📊 Using live statistics from simulation...")
    else:
        print("📊 No live stats found, using test data...")
        quality_stats = create_test_stats(total=100, rotten=40)

    # Configure and generate report
    config = ReportConfig(
        facility_name="AgriFresh Processing Plant",
        acceptable_threshold=5.0,
        critical_threshold=15.0
    )

    try:
        report = manager.create_report(quality_stats, config)
        manager.display_report()

        # Optionally save the report
        # manager.save_report("quality_report.md")

    except (ConnectionError, RuntimeError) as e:
        print(f"❌ Error generating report: {e}")


if __name__ == "__main__":
    main()

🔍 Searching for available models...
📊 No live stats found, using test data...
🤖 Consulting the AI Quality Manager...
   Processing 100 items with 40.00% loss rate...
✅ Report generated successfully!


# Quality Control Incident Report - Batch Scan Analysis

**Date:** October 26, 2023
**Prepared For:** Operations Management, Procurement Department
**Prepared By:** Quality Assurance Department
**Reference:** IoT System Data Report - Batch [Insert Batch ID/Reference]

---

## 1. Executive Summary

An automated IoT scan of a recent fruit batch revealed a significant quality issue, with a **40.00% rejection rate** due to spoilage. This unacceptably high loss rate classifies the incident as **CRITICAL**, indicating a severe deviation from quality standards and posing a substantial operational and financial impact. Immediate investigation and corrective actions are imperative to prevent further losses and maintain product quality.

---

## 2. Severity Assessment

*   **Loss Rate Identified:** 40.00%
*   **Threshold Comparison:**
    *   Acceptable: < 5.0%
    *   Warning: 5.0% - 15.0%
    *   Critical: > 15.0%
*   **Severity Level:** **CRITICAL**

**Explanation:** The observed loss rate of 40.00% far exceeds the established "Critical" threshold of >15.0%. This indicates a profound breakdown in quality control within the supply chain or handling process, necessitating urgent intervention.

---

## 3. Statistical Analysis

The IoT system scan provided the following data for the batch:

*   **Total Items Scanned:** 100 units
*   **Fresh Items:** 60 units
*   **Rotten Items (Rejections):** 40 units
*   **Calculated Loss Rate:** 40.00%

**Context:** This means that nearly half (40%) of the fruit in this specific batch was unfit for processing, directly translating to a significant reduction in usable yield and an increase in waste disposal costs. This level of spoilage is unsustainable for efficient plant operations.

---

## 4. Root Cause Analysis (Probable Causes)

Based on the high incidence of rotten items, the following are three probable causes for this critical level of spoilage:

*   **Extended Transport Delays and/or Inadequate Transit Conditions:** The fruit may have experienced prolonged transit times, especially if transportation vehicles lacked proper temperature control, ventilation, or were exposed to excessive heat/humidity. This accelerates ripening and spoilage.
*   **Improper Storage Conditions at Supplier or Receiving:** Before reaching AgriFresh, the fruit might have been stored under suboptimal conditions at the supplier's facility or immediately upon arrival at our receiving dock. Factors like high ambient temperature, excessive humidity, poor air circulation, or co-storage with ethylene-producing fruits can rapidly degrade quality.
*   **Harvest Timing and Maturity Level:** The fruit might have been harvested past its optimal ripeness window, meaning it was already over-ripe upon leaving the source. Over-ripe fruit has a significantly shorter shelf-life and is much more susceptible to bruising and spoilage during handling and transport.

---

## 5. Immediate Actions

The following steps are recommended for immediate implementation:

*   **Quarantine Remaining Material:** Immediately segregate and quarantine any remaining raw material from this specific batch or supplier awaiting processing. Do not proceed with processing until further inspection.
*   **Enhanced Incoming Inspection:** Implement a heightened, manual inspection protocol for all subsequent incoming batches, particularly from the same supplier, focusing on visual signs of spoilage, firmness, and overall quality.
*   **Review Receiving Procedures:** Conduct an immediate review of receiving protocols to ensure proper temperature checks, visual inspections, and rapid transfer to appropriate storage are being consistently applied.
*   **Supplier Communication:** Notify the supplier of the critical quality issue with photographic evidence and request details on their harvest, storage, and transport procedures for this batch.

---

## 6. Preventive Measures (Long-Term Recommendations)

To prevent recurrence of such incidents, the following long-term measures should be considered:

*   **Supplier Audit and Re-evaluation:** Conduct a comprehensive audit of the involved supplier's facilities, including harvest practices, post-harvest handling, storage conditions, and transportation logistics. Consider alternative suppliers if quality standards cannot be consistently met.
*   **Supply Chain Optimization:** Work with procurement and logistics to review and optimize transport routes, durations, and conditions. Explore options for refrigerated transport and tighter delivery schedules.
*   **Stricter Incoming Quality Specifications:** Revise and implement more stringent quality specifications for incoming raw materials, potentially including pre-shipment inspections or mandatory freshness indices.
*   **Advanced Storage Solutions:** Evaluate and invest in advanced climate-controlled storage solutions at AgriFresh to better manage inventory and extend the shelf-life of raw materials upon arrival.

---

## 7. Next Steps

*   **Cross-Departmental Meeting (EOD Today):** Schedule an urgent meeting with representatives from Operations, Procurement, and Quality Assurance to discuss this report, confirm immediate actions, and assign responsibilities.
*   **Formal Investigation (Within 24 hours):** Initiate a formal investigation into the specific batch and supplier to gather more detailed information on the root cause.
*   **Supplier Corrective Action Request (Within 48 hours):** Issue a formal Corrective Action Request (CAR) to the supplier, detailing the issue and requiring their proposed corrective and preventive actions.
*   **Monitoring and Reporting:** QA will closely monitor all incoming batches for the next two weeks and provide daily updates on quality metrics.
*   **Review of Preventive Measures (Within 1 week):** Procurement and Operations to present a plan for implementing the recommended preventive measures.

---